In [1]:
import numpy as np
import pandas as pd
import seaborn as snsn
import matplotlib.pyplot as plt
import tensorflow as tf
import os

%matplotlib inline 
#%matplotlib inline : notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해줌

In [2]:
#데이터 전처리는 기존과 동일하게 진행

# df = pd.read_csv('data/-----.csv')
df.shape

(3841, 53)

In [3]:
df.columns

Index(['Id', 'NAME', 'DOB', 'AGE', 'SEX', 'REG', 'HR', 'SDNN', 'RMSSD',
       'HRVindex', 'pNN50', 'NN50cnt', 'BeatCnt', 'ModeCnt', 'modeHR',
       'mode_RR', 'maxHR', 'minHR', 'max_minHR', 'CVAA', 'PSI', 'MSI',
       'StressIndex', 'FatigueIndex', 'BreathRate', 'HealthIndex', 'R_power',
       'LnTP', 'LnVLF', 'LnLF', 'LnHF', 'LFnorm', 'HFnorm', 'R_value', 'SD1',
       'SD2', 'rSD12', 'Ln sArea', 'mean_RR', 'Ln RMSSD', 'SDSD', 'TINN',
       'ApEn', 'Conf(%)', 'Event', 'DataLength', 'Date', 'TestTime', 'StartT',
       'EndT', 'GuestAge', 'MODESET', 'MemoCom'],
      dtype='object')

In [4]:
sample = df[['HR', 'BeatCnt', 'modeHR', 'mode_RR', 'maxHR','minHR','max_minHR','NN50cnt', 'pNN50', 'SDNN','RMSSD','Ln RMSSD','LnTP','LnVLF','LnLF','LnHF',
             'LFnorm','HFnorm','StressIndex']].copy()

In [5]:
sample.isna().sum()

HR                0
BeatCnt           0
modeHR            0
mode_RR           0
maxHR             0
minHR             0
max_minHR         0
NN50cnt           0
pNN50             0
SDNN              0
RMSSD             0
Ln RMSSD       3279
LnTP              0
LnVLF             0
LnLF              0
LnHF              0
LFnorm            0
HFnorm            0
StressIndex       0
dtype: int64

In [6]:
sample.drop(['Ln RMSSD'],axis=1, inplace=True)

In [7]:
sample.columns

Index(['HR', 'BeatCnt', 'modeHR', 'mode_RR', 'maxHR', 'minHR', 'max_minHR',
       'NN50cnt', 'pNN50', 'SDNN', 'RMSSD', 'LnTP', 'LnVLF', 'LnLF', 'LnHF',
       'LFnorm', 'HFnorm', 'StressIndex'],
      dtype='object')

In [8]:
sample.drop_duplicates(inplace=True) #중복제거
sample.shape

(3078, 18)

In [33]:
X = sample.copy()
X.drop(['StressIndex'],axis=1, inplace=True)
X.shape

(3078, 17)

In [34]:
y = sample['StressIndex'].copy()
y.shape

(3078,)

In [36]:
y

0        7
1        2
2        5
3        4
4        1
        ..
3836     2
3837    10
3838     7
3839     8
3840     5
Name: StressIndex, Length: 3078, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def make_model():
#     model = Sequential([
#         Dense(16, input_dim=1, activation='relu'),
#         Dense(16, activation='relu'),
#         Dense(10, activation='softmax')
#     ])
    model = Sequential()
    model.add(Dense(16, input_dim=1, activation='relu'))
    model.add(Dense(32,  activation='relu'))
    model.add(Dense(10,  activation='softmax'))
    
    return model


# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# mc = ModelCheckpoint('one_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [12]:
from sklearn.metrics import accuracy_score, f1_score

In [14]:
#한번에 다 돌려보자

best = []

for x in X.columns:
    print(x,' train & test')
    X[x] = X[x].values
    X_train, X_test, y_train, y_test = train_test_split(X[x], y, test_size=0.2, random_state=1)
    encoder = LabelEncoder()
    encoder.fit(y)
    y_train = encoder.transform(y_train)
    y_test = encoder.transform(y_test)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    model = make_model()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), callbacks=[es, mc])
    loaded_model = load_model('one_model.h5')
    best.append(["{} 정확도 : ".format(x), loaded_model.evaluate(X_test, y_test)[1] ])
    

HR  train & test
Epoch 1/5


NotImplementedError: in user code:

    File "C:\Users\NLP\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 953, in inner_f  *
        return f(*args, **kwargs)
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1067, in f1_score  *
        return fbeta_score(y_true, y_pred, beta=1, labels=labels,
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 953, in inner_f  *
        return f(*args, **kwargs)
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1190, in fbeta_score  *
        _, _, f, _ = precision_recall_fscore_support(y_true, y_pred,
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 953, in inner_f  *
        return f(*args, **kwargs)
    File "C:\Users\NLP\ana

In [17]:

print(len(best))
best

17


[['LnLF 정확도 : ', 0.475649356842041],
 ['LnTP 정확도 : ', 0.42694804072380066],
 ['SDNN 정확도 : ', 0.39772728085517883],
 ['LnHF 정확도 : ', 0.3782467544078827],
 ['pNN50 정확도 : ', 0.3425324559211731],
 ['NN50cnt 정확도 : ', 0.33766233921051025],
 ['RMSSD 정확도 : ', 0.32305195927619934],
 ['max_minHR 정확도 : ', 0.2792207896709442],
 ['HR 정확도 : ', 0.22240260243415833],
 ['BeatCnt 정확도 : ', 0.22240260243415833],
 ['modeHR 정확도 : ', 0.22240260243415833],
 ['mode_RR 정확도 : ', 0.22240260243415833],
 ['maxHR 정확도 : ', 0.22240260243415833],
 ['minHR 정확도 : ', 0.22240260243415833],
 ['LnVLF 정확도 : ', 0.2175324708223343],
 ['LFnorm 정확도 : ', 0.06331168860197067],
 ['HFnorm 정확도 : ', 0.06168831139802933]]

In [40]:
best.sort(key=lambda x : -x[1])
best
#이 성능은 각 컬럼 1개로만 돌려서 나온 결과임
#성능이 안좋은것부터 하나씩 빼면서 전부 돌리기

[['LnLF 정확도 : ', 0.475649356842041],
 ['LnTP 정확도 : ', 0.42694804072380066],
 ['SDNN 정확도 : ', 0.39772728085517883],
 ['LnHF 정확도 : ', 0.3782467544078827],
 ['pNN50 정확도 : ', 0.3425324559211731],
 ['NN50cnt 정확도 : ', 0.33766233921051025],
 ['RMSSD 정확도 : ', 0.32305195927619934],
 ['max_minHR 정확도 : ', 0.2792207896709442],
 ['HR 정확도 : ', 0.22240260243415833],
 ['BeatCnt 정확도 : ', 0.22240260243415833],
 ['modeHR 정확도 : ', 0.22240260243415833],
 ['mode_RR 정확도 : ', 0.22240260243415833],
 ['maxHR 정확도 : ', 0.22240260243415833],
 ['minHR 정확도 : ', 0.22240260243415833],
 ['LnVLF 정확도 : ', 0.2175324708223343],
 ['LFnorm 정확도 : ', 0.06331168860197067],
 ['HFnorm 정확도 : ', 0.06168831139802933]]

In [174]:
#컬럼수 17
from tensorflow.keras.layers import Dropout

dropout = 0.3
# input_columns=4

def model(input_columns):
    model = Sequential([
        Dense(64, input_dim=input_columns, activation='relu'),
        Dense(128),
        Dense(256, activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    return model

In [160]:
X.columns

Index(['SDNN', 'LnTP', 'LnLF'], dtype='object')

In [77]:
y

0        7
1        2
2        5
3        4
4        1
        ..
3836     2
3837    10
3838     7
3839     8
3840     5
Name: StressIndex, Length: 3078, dtype: int64

In [80]:
y.iloc[3,:] #y가 시리즈이고 열이 하나뿐인데 ',' 를 사용하니까 문제가 생김

IndexingError: Too many indexers

In [162]:
X = sample.copy()
X.drop(['StressIndex'],axis=1, inplace=True) #17
X.drop(['HFnorm'], axis=1, inplace=True) #16
X.drop(['LFnorm'], axis=1, inplace=True) #15
X.drop(['LnVLF'], axis=1, inplace=True) #14
X.drop(['minHR'], axis=1, inplace=True) #13
X.drop(['maxHR'], axis=1, inplace=True) #12
X.drop(['mode_RR'], axis=1, inplace=True) #11
X.drop(['modeHR'], axis=1, inplace=True) #10
X.drop(['BeatCnt'], axis=1, inplace=True) #9
X.drop(['HR'], axis=1, inplace=True) #8
X.drop(['max_minHR'], axis=1, inplace=True) #7
X.drop(['RMSSD'], axis=1, inplace=True) #6
X.drop(['NN50cnt'], axis=1, inplace=True) #5
X.drop(['pNN50'], axis=1, inplace=True) #4
# X.drop(['LnHF'], axis=1, inplace=True) #3

In [163]:
X.columns

Index(['SDNN', 'LnTP', 'LnLF', 'LnHF'], dtype='object')

In [252]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score




# y = sample['StressIndex'].copy()




# num_folds=5

# kfold = KFold(n_splits=num_folds, shuffle=True)

# kfold에 진짜 진짜 중요한 거! 밑에 코드에서 train과 test에는 index가 옴!! 데이터가 넘어간다고 생각을 하면 이해못함
# 괜히 구글링해서 보는 코드들이 변수를 ~_idx라고 쓰는게 아니다. 이걸 생각하면 코드를 어떻게 짜야 할지 이해가 쉬움. 일단 텐서플로우는 완벽히 이해 됬고 파이토치도 다시 확인해보자.
# 파이토치 코드 finalproject/final-20-nd...
# kfold.split(X), kfold.split(X,y) 현재는 둘다 상관 없다. 어차피 밑에서 모델링 할 때 one-hot encoding을 해주기도 하고 인덱스 번호는 X로도 충분하며, X와 y의 개수는 같기 때문이다.


score_acc = []
score_loss = []
def kfold_func(kfold,X, y):
    for fold,(train_idx, test_idx) in enumerate(kfold.split(X,y)):
    #     print(train_idx, len(train_idx))
    #     print(test_idx, len(test_idx))
        model2 = model(3)
        model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        encoder2 = LabelEncoder()
        encoder2.fit(y)

        X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
#         X_train, X_test = X.iloc[train_idx], X.iloc[test_idx] #feature 1개만 사용시
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx] 

        y_train = encoder2.transform(y_train)
        y_test = encoder2.transform(y_test)
        y_train = to_categorical(y_train)
        y_test = to_categorical(y_test)


        #새로 알게된 사실! 이녀석도 같이 돌아야함. 단지 콜백함수로써 작동만 하는 함수인줄 알았는데 이걸 밖에 선언하면 기존 val_loss를 기억하고 있음...
        #(model을 위에서 새로 선언해주었지만 전 model의 최소 loss를 가지고 있다는 말)
    #     es = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='min')
        mc = ModelCheckpoint(filepath='0_model.h5',monitor='val_loss', mode='min', verbose=1, save_best_only=True)

        print('------------------------------------------------------')
        print('Training for {}'.format(fold))
        history = model2.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_test, y_test), callbacks=[mc])
        loaded_model = load_model('0_model.h5')
        score_acc.append(loaded_model.evaluate(X_test, y_test)[1])
        score_loss.append(loaded_model.evaluate(X_test, y_test)[0])



    #이거는 list가 리턴된다. 처음은 loss, 두번째는 acc
    # loaded_model.evaluate(X_test, y_test)     

    print(f'최종 평균 accuracy_score : {sum(score_acc)/len(score_acc)}')
    print(f'최종 평균 loss_score : {sum(score_loss)/len(score_loss)}')

17_model.h5 (17 feautures)
 최종 평균 accuracy_score : 0.4806504130363464
 최종 평균 loss_score : 1.4827069282531737 <br>
 
16_model.h5
최종 평균 accuracy_score : 0.4634146332740784
최종 평균 loss_score : 1.5211002588272096 <br>

15_model.h5
최종 평균 accuracy_score : 0.39284552335739137
최종 평균 loss_score : 1.658779215812683<br>

14_model.h5
최종 평균 accuracy_score : 0.41203251481056213
최종 평균 loss_score : 1.6400482416152955<br>

13_model.h5
최종 평균 accuracy_score : 0.3925203263759613
최종 평균 loss_score : 1.6505762815475464<br>

12_model.h5
최종 평균 accuracy_score : 0.4039024353027344
최종 평균 loss_score : 1.6676501035690308<br>

11_model.h5
최종 평균 accuracy_score : 0.44455283880233765
최종 평균 loss_score : 1.5161062479019165<br>

10_model.h5
최종 평균 accuracy_score : 0.4520325243473053
최종 평균 loss_score : 1.5067994356155396<br>

9_model.h5
최종 평균 accuracy_score : 0.49008129835128783
최종 평균 loss_score : 1.434326457977295<br>

8_model.h5
최종 평균 accuracy_score : 0.5050406396389008
최종 평균 loss_score : 1.389447283744812 <br>

7_model.h5
최종 평균 accuracy_score : 0.534959352016449
최종 평균 loss_score : 1.2804981231689454<br>

6_model.h5
최종 평균 accuracy_score : 0.5655284523963928
최종 평균 loss_score : 1.225916814804077<br>

5_model.h5
최종 평균 accuracy_score : 0.600000011920929
최종 평균 loss_score : 1.1347815990447998<br>

4_model.h5
최종 평균 accuracy_score : 0.6478048682212829
최종 평균 loss_score : 1.0561283349990844<br>

3_model.h5
최종 평균 accuracy_score : 0.5365853786468506
최종 평균 loss_score : 1.3134488821029664<br>

'SDNN', 'LnTP', 'LnLF', 'LnHF' 4개가 동일한 배경에서 가장 좋은 성능을 보여줌<br>
epochs=100 으로 설정시 <br>
최종 평균 accuracy_score : 0.7053658366203308
최종 평균 loss_score : 0.9086637616157531

## 통계 features
### SDNN, RMSSD, NN50cnt, pNN50

In [167]:
sample.columns

Index(['HR', 'BeatCnt', 'modeHR', 'mode_RR', 'maxHR', 'minHR', 'max_minHR',
       'NN50cnt', 'pNN50', 'SDNN', 'RMSSD', 'LnTP', 'LnVLF', 'LnLF', 'LnHF',
       'LFnorm', 'HFnorm', 'StressIndex'],
      dtype='object')

In [188]:
X = sample[['SDNN']].copy()
y = sample['StressIndex'].copy()

In [209]:
kfold = KFold(n_splits=5, shuffle=True)
kfold_func(kfold, X, y)

------------------------------------------------------
Training for 0
Epoch 1/40
70/77 [==========================>...] - ETA: 0s - loss: 2.0104 - accuracy: 0.3009
Epoch 1: val_loss improved from inf to 1.70500, saving model to 0_model.h5
77/77 [==============================] - 1s 13ms/step - loss: 1.9795 - accuracy: 0.3163 - val_loss: 1.7050 - val_accuracy: 0.3642
Epoch 2/40
74/77 [===========================>..] - ETA: 0s - loss: 1.5606 - accuracy: 0.4426
Epoch 2: val_loss improved from 1.70500 to 1.53577, saving model to 0_model.h5
77/77 [==============================] - 1s 13ms/step - loss: 1.5545 - accuracy: 0.4446 - val_loss: 1.5358 - val_accuracy: 0.4081
Epoch 3/40
68/77 [=========================>....] - ETA: 0s - loss: 1.4357 - accuracy: 0.4945
Epoch 3: val_loss improved from 1.53577 to 1.41624, saving model to 0_model.h5
77/77 [==============================] - 1s 11ms/step - loss: 1.4431 - accuracy: 0.4868 - val_loss: 1.4162 - val_accuracy: 0.4846
Epoch 4/40
65/77 [=======

KeyboardInterrupt: 

SDNN, RMSSD, NN50cnt, pNN50 <br>
최종 평균 accuracy_score : 0.41008129715919495<br>
최종 평균 loss_score : 1.6236515283584594 <br>


SDNN, NN50cnt, pNN50<br>
최종 평균 accuracy_score : 0.4100813001394272 <br>
최종 평균 loss_score : 1.6175793886184693 <br>

SDNN, pNN50<br>
최종 평균 accuracy_score : 0.40813007950782776 <br>
최종 평균 loss_score : 1.6006160259246827 <br>

SDNN<br>
최종 평균 accuracy_score : 0.40195122361183167<br>
최종 평균 loss_score : 1.6414166688919067<br>

SDNN 혼자 다함

## Frequency-Domain features
### LnTP, LnVLF, LnLF, LnHF, LFnorm, HFnorm

In [251]:
X = sample[['LnLF','LnHF','SDNN']].copy()
y = sample['StressIndex'].copy()

In [253]:
kfold = KFold(n_splits=5, shuffle=True)
kfold_func(kfold, X, y)

------------------------------------------------------
Training for 0
Epoch 1/40
71/77 [==========================>...] - ETA: 0s - loss: 2.6142 - accuracy: 0.1928
Epoch 1: val_loss improved from inf to 2.31888, saving model to 0_model.h5
77/77 [==============================] - 1s 10ms/step - loss: 2.5732 - accuracy: 0.2011 - val_loss: 2.3189 - val_accuracy: 0.2711
Epoch 2/40
70/77 [==========================>...] - ETA: 0s - loss: 2.1270 - accuracy: 0.2205
Epoch 2: val_loss improved from 2.31888 to 1.96535, saving model to 0_model.h5
77/77 [==============================] - 1s 10ms/step - loss: 2.1167 - accuracy: 0.2271 - val_loss: 1.9654 - val_accuracy: 0.2825
Epoch 3/40
77/77 [==============================] - ETA: 0s - loss: 1.9302 - accuracy: 0.3152
Epoch 3: val_loss did not improve from 1.96535
77/77 [==============================] - 0s 5ms/step - loss: 1.9302 - accuracy: 0.3152 - val_loss: 1.9927 - val_accuracy: 0.3166
Epoch 4/40
77/77 [==============================] - ETA: 0

LnTP, LnVLF, LnLF, LnHF, LFnorm, HFnorm<br>
최종 평균 accuracy_score : 0.7313182353973389<br>
최종 평균 loss_score : 0.8469133615493775<br>

LnTP, LnVLF, LnLF, LnHF, LFnorm <br>
최종 평균 accuracy_score : 0.7066313028335571<br>
최종 평균 loss_score : 0.9142210602760314<br>

LnTP, LnVLF, LnLF, LnHF<br>
최종 평균 accuracy_score : 0.7303526639938355<br>
최종 평균 loss_score : 0.8638480544090271<br>

LnTP, LnLF, LnHF<br>
최종 평균 accuracy_score : 0.7361989259719849
최종 평균 loss_score : 0.8622489929199219

LnTP, LnLF<br>
최종 평균 accuracy_score : 0.604284131526947
최종 평균 loss_score : 1.1445185661315918

LnTP <br>
최종 평균 accuracy_score : 0.5079703748226165
최종 평균 loss_score : 1.3475021123886108

LnLF<br>
최종 평균 accuracy_score : 0.5238346854845682<br>
최종 평균 loss_score : 1.3154326836268107<br>

LnHF <br>
최종 평균 accuracy_score : 0.5532963633537292 <br>
최종 평균 loss_score : 1.2522465944290162 <br>

LnLF, LnHF<br>
최종 평균 accuracy_score : 0.7576459765434265<br>
최종 평균 loss_score : 0.8495001673698426<br>

In [17]:
len(X.columns)

17

In [27]:
y

0        7
1        2
2        5
3        4
4        1
        ..
3836     2
3837    10
3838     7
3839     8
3840     5
Name: StressIndex, Length: 3078, dtype: int64

In [37]:
f1model = Sequential([
    Dense(64, input_dim=17, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax'),
])

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [39]:
encoder = LabelEncoder()
encoder.fit(y)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [40]:
y_test

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [42]:
f1model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_score])
history = f1model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5


NotImplementedError: in user code:

    File "C:\Users\NLP\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 953, in inner_f  *
        return f(*args, **kwargs)
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1067, in f1_score  *
        return fbeta_score(y_true, y_pred, beta=1, labels=labels,
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 953, in inner_f  *
        return f(*args, **kwargs)
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1190, in fbeta_score  *
        _, _, f, _ = precision_recall_fscore_support(y_true, y_pred,
    File "C:\Users\NLP\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 953, in inner_f  *
        return f(*args, **kwargs)
    File "C:\Users\NLP\ana

In [112]:
y_pred = f1model.predict(X_train)

77/77 [==============================] - 0s 2ms/step


In [113]:
y_pred

array([[5.69635129e-04, 2.90208776e-02, 1.20577283e-01, ...,
        7.03986984e-07, 8.85614834e-04, 1.27365429e-05],
       [8.54957732e-04, 2.78629251e-02, 6.29631476e-03, ...,
        1.24702467e-06, 1.20369084e-04, 2.02332274e-07],
       [4.22280536e-05, 1.39996829e-02, 8.60592946e-02, ...,
        4.36922011e-04, 1.14175033e-04, 1.54377412e-05],
       ...,
       [2.85563040e-09, 1.23474596e-03, 6.85422623e-04, ...,
        1.76999113e-03, 3.35746147e-02, 5.11190474e-01],
       [8.85633126e-05, 5.93443960e-03, 1.01536639e-01, ...,
        7.10301974e-05, 3.08263006e-05, 3.83686893e-06],
       [1.18331191e-05, 3.27789545e-04, 8.62231129e-04, ...,
        6.60700243e-06, 1.47938135e-03, 4.26721163e-05]], dtype=float32)

In [129]:
y_pred[0]

array([5.6963513e-04, 2.9020878e-02, 1.2057728e-01, 2.9972295e-04,
       7.5673723e-01, 9.1880247e-02, 1.5966885e-05, 7.0398698e-07,
       8.8561483e-04, 1.2736543e-05], dtype=float32)

In [117]:
a = np.where(np.argmax(y_pred[0], axis=0), 1, 0)

In [119]:
np.argmax(y_pred[0])

4

In [124]:
y_pred[0,1]

0.029020878

In [127]:
a = np.where(y_pred[0,4],1, 0)

In [128]:
a

array(1)

In [130]:
#이거 실행 시켜야 됨
#이거 안시키면 y_pred랑 y_true랑 비교가 안됨 숫자가 다름
for row in range(0,2462):
    for columns in range(0,10): 
        index = np.argmax(y_pred[row])
        if columns == index : 
            y_pred[row, columns] = 1
        else:
            y_pred[row, columns] = 0
        

In [132]:
y_pred[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [133]:
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score
# metrics.classification_report(y_train, y_pred, digits=3)
f1_score(y_train, y_pred, average='weighted') #test와 

0.22431035287587722

In [70]:
a = np.array([[15, 8, 12], [11, 7, 3]])
b = np.where(a > 10, a, 10)

In [71]:
b

array([[15, 10, 12],
       [11, 10, 10]])

In [91]:
length=2462

for x in y_pred:
    i=0
    max =x[0]
    if max < x[i]:
        
    

[5.6963513e-04 2.9020878e-02 1.2057728e-01 2.9972295e-04 7.5673723e-01
 9.1880247e-02 1.5966885e-05 7.0398698e-07 8.8561483e-04 1.2736543e-05]
